<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_p3_deeplabv3plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [3]:
import os


train_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot'
img_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.jpg')])

In [4]:
from PIL import Image


class LandDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        return img, os.path.basename(self.img_paths[idx])

In [5]:
from torchvision import transforms as tr

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

test_ds = LandDataset(img_paths_train,
                       transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                      )

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
from torchvision.models.segmentation.deeplabv3 import deeplabv3_resnet101, DeepLabHead, FCNHead


def deeplabv3(mode, num_classes=7):
    if mode == 'resnet101':
        model =  deeplabv3_resnet101(pretrained=True)
        model.aux_classifier = FCNHead(1024, num_classes)
        model.classifier = DeepLabHead(2048, num_classes)
    return model

def get_model(path):
    model = deeplabv3(mode='resnet101')
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    model.eval()
    return model

In [7]:
import numpy as np
import imageio


def predimg(preds, names, out_path):
    for pred, name in zip(preds, names):
        pred = pred.detach().cpu().numpy()
        pred_img = np.zeros((512, 512, 3), dtype=np.uint8)
        pred_img[np.where(pred == 0)] = [0, 255, 255]
        pred_img[np.where(pred == 1)] = [255, 255, 0]
        pred_img[np.where(pred == 2)] = [255, 0, 255]
        pred_img[np.where(pred == 3)] = [0, 255, 0]
        pred_img[np.where(pred == 4)] = [0, 0, 255]
        pred_img[np.where(pred == 5)] = [255, 255, 255]
        pred_img[np.where(pred == 6)] = [0, 0, 0]
        new_name = name.replace('sat', 'mask').replace('.jpg', '.png')
        imageio.imwrite(os.path.join(out_path, new_name), pred_img)

In [9]:
epoch_1 = get_model('/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/epoch1model.pth')
epoch_40 = get_model('/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/epoch40model.pth')
epoch_80 = get_model('/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/BestIOU.pth')

output_directorys = ['/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot/first/',
                    '/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot/mid/',
                    '/content/drive/MyDrive/NTU_DLCV/p3_data/train_plot/last/']

In [14]:
for model, output_directory in zip([epoch_1, epoch_40, epoch_80], output_directorys):
    try:
        os.makedirs(output_directory, exist_ok=True)
    except:
        pass
    for x, filenames in test_loader:
        with torch.no_grad():
            x = x.to(device)
            pred_out = model(x)['out']
        pred_out = pred_out.argmax(dim=1)
        predimg(pred_out, filenames, output_directory)